In [1]:
DATA_FOLDER = 'data'
TRAIN_FILE = 'dataset.csv'
PRED_FILE = 'test_dataset.csv'

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
import os
import re
import string
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import spacy.attrs
import pickle 


from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin, clone
from tqdm.notebook import tqdm




In [4]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("merge_entities", after="ner")
en_stopwords = nlp.Defaults.stop_words

<function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>

In [5]:
cpu_count = min(int(os.cpu_count()), 4) if os.cpu_count() != None else 4

In [6]:
TRAIN_PATH = os.path.join(DATA_FOLDER, TRAIN_FILE)
PRED_PATH = os.path.join(DATA_FOLDER, PRED_FILE)

In [7]:
TRAIN_PATH
PRED_PATH

'data/dataset.csv'

'data/test_dataset.csv'

In [8]:
data = pd.read_csv(TRAIN_PATH)

In [9]:
def convert_series2docs(nlp_, data_: pd.Series, num_processes: int):
    docs_ = list(tqdm(nlp_.pipe(data_, batch_size=20, n_process=num_processes), total=len(data_)))
    return docs_

In [10]:
docs = convert_series2docs(nlp, data['reviewText'], cpu_count-1)

KeyError: 'reviewText'

In [11]:
def dataframe_analysis(df_: pd.DataFrame):
    info_df = df_.info()
    describe_df = df_.describe()
    value_counts_df_ = [df_[column].value_counts() for column in df_.columns]
    return info_df, describe_df, value_counts_df_

In [12]:
dataframe_analysis(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9375 entries, 0 to 9374
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      9373 non-null   object
 1   date      9373 non-null   object
 2   category  9373 non-null   object
 3   language  9373 non-null   object
dtypes: object(4)
memory usage: 293.1+ KB


(None,
                                        text       date  \
 count                                  9373       9373   
 unique                                 9371        331   
 top     Your focus determines your reality.  10-3-2022   
 freq                                      2         46   
 
                                    category language  
 count                                  9373     9373  
 unique                                   78        2  
 top     direct_debit_payment_not_recognised       en  
 freq                                    174     9073  ,
 [Your focus determines your reality.                                                                                                                                                                   2
  Boa noite, Como faço para que o meu cartão apareça na aplicação? Cumprimentos                                                                                                                         2
  Olá, Poss